In [1]:
from pathlib import Path

import pandas as pd

import numpy as np

import meam2nn

Define function for comparing LAMMPS parameter files

In [2]:
def compare_parameters(params1, params2, ignore_missing=None, ignore_different=None, rtol=1e-05, atol=1e-08):
    """
    Compares MEAM parameters from two parameter files and identifies the parameters that
    are missing from one or the other files and the paramters that have different values.
    
    Parameters
    ----------
    params1 : dict
        The parameter file parameters from one of the files to compare.
    params2 : dict
        The parameter file parameters from one of the files to compare.
    ignore_missing : list, optional
        Parameter keys to ignore if missing from one of the files.
    ignore_different : list, optional
        Parameter keys to ignore if the value is different between the files.
    rtol : float, optional
        The relative tolerance to use in comparing numerical values.  Default value is 1e-5.
    atol : float, optional
        The absolute tolerance to use in comparing numerical values.  Default value is 1e-8.
    """
    
    missing1 = []
    missing2 = []
    different = []
    if ignore_missing is None:
        ignore_missing = []
    if ignore_different is None:
        ignore_different = []
    
    for key in params1:
        if key not in params2:
            if key not in ignore_missing:
                missing2.append(key)
        else:
            try:
                if not np.isclose(params1[key], params2[key], rtol=rtol, atol=atol) and key not in ignore_different:
                    different.append(key)
            except:
                if params1[key] != params2[key]:
                    different.append(key)                
        
    for key in params2:
        if key not in params1:
            if key not in ignore_missing:
                missing1.append(key)
    
    if len(missing1) > 0:
        print('First parameter set is missing:')
        for key in missing1:
            print('-', key)
    
    if len(missing2) > 0:
        print('Second parameter set is missing:')
        for key in missing2:
            print('-', key)
            
    if len(different) > 0:
        print('Different values are:')
        for key in different:
            print('-', key)
            print('  *', params1[key])
            print('  *', params2[key])
            
    if len(missing1) == 0 and len(missing2) == 0 and len(different) == 0:
        print('All values match!')

Search for LAMMPS files

In [3]:
library_files = []
parameter_files = []

for fname in Path('LAMMPS').glob('*/*/*.meam'):
    if fname.name == 'library.meam':
        library_files.append(fname)
    else:
        parameter_files.append(fname)

Compare parameter file values between csv files and LAMMPS \*.meam files 

In [9]:
db = meam2nn.MEAM2NN()

ignore_missing = ['mixture_ref_t', 'emb_lin_neg', 'bkgd_dyn', 'alpha(1,1)', 'alpha(2,2)', 'alpha(3,3)']
ignore_different = ['rc']

for paramfile in parameter_files:
    
    # Extract names of elements and order from the file name
    elementstr = paramfile.stem.replace('-para', '')
    uppers = []
    for i in range(len(elementstr)):
        if elementstr[i].isupper():
            uppers.append(i)
    elements = []
    for i in range(len(uppers)):
        s = uppers[i]
        try:
            e = uppers[i+1]
        except:
            elements.append(elementstr[s:])
        else:
            elements.append(elementstr[s:e])

    print(paramfile)
    params2 = meam2nn.parse_lammps_parameter_file(paramfile)
    
    # Identify all symbol models associated with the elements
    allsymbols = []
    if len(elements) == 1:
        matches = db.elements[db.elements.El == elements[0]]
        for index in matches.index:
            allsymbols.append([matches.loc[index].Sym])
    elif len(elements) == 2:
        matches = db.alloys[((db.alloys.El1==elements[0]) & (db.alloys.El2==elements[1]))]
        for index in matches.index:
            allsymbols.append([matches.loc[index].Sym1, matches.loc[index].Sym2])
    elif len(elements) == 3:
        matches = db.ternaries[((db.ternaries.El1==elements[0]) & (db.ternaries.El2==elements[1]) & (db.ternaries.El3==elements[2]))]
        for index in matches.index:
            allsymbols.append([matches.loc[index].Sym1, matches.loc[index].Sym2, matches.loc[index].Sym3])
    else:
        print('Skipping >3 elements for now...')
    
    if len(allsymbols) == 0:
        print('No matching parameters found!')
        
    # Loop over all matching symbols and compare
    for symbols in allsymbols:
        try:
            params1 = meam2nn.parse_lammps_parameter_file(db.lammps_parameter(symbols))
        except:
            pass
        else:
            print(symbols)
            compare_parameters(params1, params2, ignore_missing=ignore_missing, ignore_different=ignore_different)
    print()

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Cr\1\Cr.meam
['Cr']
All values match!
['CR']
Different values are:
- repuls(1,1)
  * 0.1
  * 0.0
- attrac(1,1)
  * 0.02
  * 0.0
- Cmin(1,1,1)
  * 0.71
  * 0.78

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Fe\1\Fe.meam
['Fe']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Mo\1\Mo.meam
['Mo']
All values match!
['MO']
Different values are:
- Cmin(1,1,1)
  * 0.82
  * 0.64
- Cmax(1,1,1)
  * 2.5
  * 2.8

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Nb\1\Nb.meam
['Nb']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--Ta\1\Ta.meam
['Ta']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--V\1\V.meam
['V']
All values match!

LAMMPS\2001--Lee-B-J-Baskes-M-I-Kim-H-Cho-Y-K--W\1\W.meam
['W']
All values match!

LAMMPS\2001--Lee-B-J-Shim-J-H-Park-H-M--Fe-Cr\1\FeCr.meam
['Fe', 'Cr']
Different values are:
- alpha(1,2)
  * 5.42999495353109
  * 5.42992091152959
- Cmin(1,2,1)
  * 0.55
  * 0.54995282598983

  * 0.5625
  * 0.56
- Cmin(2,1,2)
  * 0.5625
  * 0.56
['Ni', 'Fe']
Different values are:
- Cmin(1,2,1)
  * 0.5625
  * 0.56
- Cmin(1,2,2)
  * 0.5625
  * 0.56
- Cmin(2,1,1)
  * 0.5625
  * 0.56
- Cmin(2,1,2)
  * 0.5625
  * 0.56

LAMMPS\2018--Choi-W-M-Jo-Y-H-Sohn-S-S-et-al--Co-Ni-Cr-Fe-Mn\1\CoNiCrFeMn.meam
Skipping >3 elements for now...
No matching parameters found!

LAMMPS\2018--Jang-H-S-Kim-K-M-Lee-B-J--Mg-Nd\1\MgNd.meam
['Mg', 'Nd']
All values match!

LAMMPS\2018--Jang-H-S-Kim-K-M-Lee-B-J--Mg-Pb\1\MgPb.meam
['Mg', 'Pb']
All values match!

LAMMPS\2018--Jang-H-S-Kim-K-M-Lee-B-J--Zn\1\Zn.meam
['Zn']
Second parameter set is missing:
- ialloy
- rho0(1)

LAMMPS\2018--Jang-H-S-Kim-K-M-Lee-B-J--Zn-Mg\1\ZnMg.meam
['Zn', 'Mg']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Al\1\PdAl.meam
['Pd', 'Al']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Co\1\PdCo.meam
['Pd', 'Co']
All values match!

LAMMPS\2018--Jeong-G-U-Park-C-S-Do-H-S-et-al--Pd-Cu\1\PdCu.mea

In [5]:
elements

['Cu', 'Mo']

In [6]:
symbols

['Cu', 'MO']

In [7]:
params1

NameError: name 'params1' is not defined